# Thu thập dữ liệu cầu thủ từ Web

---
## Import thư viện cần thiết

In [20]:
import numpy as np
import pandas as pd 
from pandas import DataFrame
import re
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [22]:
rq = requests.get("https://sofifa.com/robots.txt")
print(rq.text)

User-agent: *
Disallow: /players/compare


---
## Click chọn các mục cần thiết từ trang web

In [ ]:
# Khởi tạo web driver.
browser = webdriver.Firefox(executable_path='./geckodriver.exe')
browser.get("https://sofifa.com/players?offset=0")
time.sleep(10)

# Tìm dropdown button
specials = browser.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[1]/form/div/div[1]/div/div[1]/input')
# Click dropdown button
specials.click()
time.sleep(2)
#Tìm id của dropdownlist ( id được random mỗi lần)
col_id = browser.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[1]/form/div/div[1]/div/div[2]/div/div[2]')
col_id.get_attribute("id")
id = col_id.get_attribute("id")
id = id[0:len(id) - 1]
time.sleep(5)

xpathid = '//*[@id="'+id+'58'+'"]'
base_stats = browser.find_element_by_xpath(xpathid)
base_stats.click()
time.sleep(5)

specials = browser.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[1]/form/div/div[1]/div/div[1]/input')
specials.click()
time.sleep(2)

xpathid = '//*[@id="'+id+'63'+'"]'
InternationalReputation = browser.find_element_by_xpath(xpathid)
InternationalReputation.click()
time.sleep(5)

specials = browser.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[1]/form/div/div[1]/div/div[1]/input')
specials.click()
time.sleep(2)

xpathid = '//*[@id="'+id+'64'+'"]'
PAC = browser.find_element_by_xpath(xpathid)
PAC.click()
time.sleep(5)

specials = browser.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[1]/form/div/div[1]/div/div[1]/input')
specials.click()
time.sleep(2)

xpathid = '//*[@id="'+id+'65'+'"]'
SHO = browser.find_element_by_xpath(xpathid)
SHO.click()
time.sleep(5)


specials = browser.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[1]/form/div/div[1]/div/div[1]/input')
specials.click()
time.sleep(2)

xpathid = '//*[@id="'+id+'66'+'"]'
PAS = browser.find_element_by_xpath(xpathid)
PAS.click()
time.sleep(5)


specials = browser.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[1]/form/div/div[1]/div/div[1]/input')
specials.click()
time.sleep(2)

xpathid = '//*[@id="'+id+'67'+'"]'
DRI = browser.find_element_by_xpath(xpathid)
DRI.click()
time.sleep(5)


specials = browser.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[1]/form/div/div[1]/div/div[1]/input')
specials.click()
time.sleep(2)

xpathid = '//*[@id="'+id+'68'+'"]'
DEF = browser.find_element_by_xpath(xpathid)
DEF.click()
time.sleep(5)

specials = browser.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[1]/form/div/div[1]/div/div[1]/input')
specials.click()
time.sleep(2)

xpathid = '//*[@id="'+id+'69'+'"]'
PHY = browser.find_element_by_xpath(xpathid)
PHY.click()
time.sleep(5)


In [ ]:
# Cancel dropdownlist vì list này chặn button apply
cancelpopup = browser.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/div/table/thead/tr[1]/th[2]')
cancelpopup.click()
time.sleep(2)
apply = browser.find_element_by_xpath('/html/body/div[1]/div/div/div[2]/div[1]/form/div/div[2]/button')
apply.click()
time.sleep(2)

---
## Thu thập dữ liệu

In [ ]:
columns = ['ID','Name','Position', 'Overall', 'Potential','BS','IR',
          'PAC','SHO','PAS','DRI','DEF','PHY','Value', 'Wage','TotalStats']

data = DataFrame(columns = columns) # Tạo DataFrame có các cột như trên
offset = 0;
while True: # Duyệt các page
    url ="https://sofifa.com/?col=vl&sort=desc&offset=%s" % str(offset*60)
    browser.get(url)
    while("No results found." in browser.page_source or 'Oops! This Page Could Not Be Found' in browser.page_source):
        browser.get(url)
        print('Can not find page')
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    table_body = soup.find('tbody') # Lấy thẻ tbody
    i=0
    for row in table_body.findAll('tr'): # Duyệt các thẻ tr con
        # Mỗi thẻ tr chứa thông tin của 1 cầu thủ
        td = row.findAll('td')
        ID=td[0].find('img').get('id')
        pos_array = td[1].findAll('a',recursive=False)
        count = 0
        str_pos = []
        for pos in pos_array:
            count = count + 1
            if count == 1:
                continue
            str_pos.append(pos.text)
        position = ' '.join(str_pos)
        name=row.find('div',{'class':'bp3-text-overflow-ellipsis'}).text
        overall = td[3].text.strip() # Lấy chỉ số overall
        potential = td[4].text.strip()        
        value = td[6].text.strip() # Lấy giá trị cầu thủ và loại bỏ \n
        if value == '€0' or value == '$0' or value == '0' or offet >= 320:
            break
        wage=td[7].text.strip() # Lấy lương
        totalStats=td[8].text.strip() 
        BS=td[9].text.strip()
        IR=td[10].text.strip()
        PAC=td[11].text.strip()
        SHO=td[12].text.strip()
        PAS=td[13].text.strip()
        DRI=td[14].text.strip()
        DEF=td[15].text.strip()
        PHY=td[16].text.strip()
        # Tạo DataFrame từ các attribute đã có
        player_data = DataFrame([[ID,name,position,overall,potential,BS,IR,PAC,SHO,PAS,DRI,DEF,PHY,value,wage,totalStats]])
        player_data.columns = columns # Đặt tên các cột
        data = data.append(player_data, ignore_index=True) # Thêm nó vào data
    offset+=1 # Xét trang tiếp theo


In [ ]:
# Xóa các dòng có ID của cầu thủ trùng nhau
data.drop_duplicates(subset='ID')

In [ ]:
browser.quit()

---
## Ghi ra file

In [ ]:
data.to_csv('final_data.csv',encoding="utf-8-sig")